# Mécanismes d'attention et transformateurs

Un inconvénient majeur des réseaux récurrents est que tous les mots d'une séquence ont le même impact sur le résultat. Cela entraîne des performances sous-optimales avec les modèles standard encodeur-décodeur LSTM pour les tâches de séquence à séquence, telles que la reconnaissance d'entités nommées et la traduction automatique. En réalité, certains mots spécifiques de la séquence d'entrée ont souvent plus d'impact sur les sorties séquentielles que d'autres.

Prenons un modèle de séquence à séquence, comme la traduction automatique. Il est implémenté par deux réseaux récurrents, où un réseau (**encodeur**) compresse la séquence d'entrée dans un état caché, et un autre (**décodeur**) déploie cet état caché pour produire le résultat traduit. Le problème avec cette approche est que l'état final du réseau a du mal à se souvenir du début de la phrase, ce qui entraîne une qualité médiocre du modèle pour les phrases longues.

Les **mécanismes d'attention** offrent un moyen de pondérer l'impact contextuel de chaque vecteur d'entrée sur chaque prédiction de sortie du RNN. Cela est réalisé en créant des raccourcis entre les états intermédiaires du RNN d'entrée et du RNN de sortie. Ainsi, lors de la génération du symbole de sortie $y_t$, nous prenons en compte tous les états cachés d'entrée $h_i$, avec différents coefficients de pondération $\alpha_{t,i}$.

![Image montrant un modèle encodeur/décodeur avec une couche d'attention additive](../../../../../lessons/5-NLP/18-Transformers/images/encoder-decoder-attention.png)
*Le modèle encodeur-décodeur avec mécanisme d'attention additive dans [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), cité de [ce billet de blog](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

La matrice d'attention $\{\alpha_{i,j}\}$ représente le degré auquel certains mots d'entrée influencent la génération d'un mot donné dans la séquence de sortie. Voici un exemple de cette matrice :

![Image montrant un alignement trouvé par RNNsearch-50, tiré de Bahdanau - arviz.org](../../../../../lessons/5-NLP/18-Transformers/images/bahdanau-fig3.png)

*Figure tirée de [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Les mécanismes d'attention sont responsables de la plupart des avancées actuelles ou proches de l'état de l'art en traitement du langage naturel. Cependant, l'ajout d'attention augmente considérablement le nombre de paramètres du modèle, ce qui a entraîné des problèmes de mise à l'échelle avec les RNN. Une contrainte clé pour la mise à l'échelle des RNN est que la nature récurrente des modèles rend difficile le traitement par lots et la parallélisation de l'entraînement. Dans un RNN, chaque élément d'une séquence doit être traité dans un ordre séquentiel, ce qui signifie qu'il ne peut pas être facilement parallélisé.

L'adoption des mécanismes d'attention combinée à cette contrainte a conduit à la création des modèles transformateurs, désormais à l'état de l'art, que nous connaissons et utilisons aujourd'hui, de BERT à OpenGPT3.

## Modèles transformateurs

Au lieu de transmettre le contexte de chaque prédiction précédente à l'étape d'évaluation suivante, les **modèles transformateurs** utilisent des **encodages positionnels** et l'attention pour capturer le contexte d'une entrée donnée dans une fenêtre de texte fournie. L'image ci-dessous montre comment les encodages positionnels avec attention peuvent capturer le contexte dans une fenêtre donnée.

![GIF animé montrant comment les évaluations sont effectuées dans les modèles transformateurs.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Étant donné que chaque position d'entrée est mappée indépendamment à chaque position de sortie, les transformateurs peuvent mieux paralléliser que les RNN, ce qui permet des modèles de langage beaucoup plus grands et plus expressifs. Chaque tête d'attention peut être utilisée pour apprendre différentes relations entre les mots, ce qui améliore les tâches de traitement du langage naturel en aval.

**BERT** (Bidirectional Encoder Representations from Transformers) est un réseau transformateur multi-couches très large avec 12 couches pour *BERT-base* et 24 pour *BERT-large*. Le modèle est d'abord pré-entraîné sur un corpus de texte volumineux (WikiPedia + livres) en utilisant un entraînement non supervisé (prédiction des mots masqués dans une phrase). Pendant le pré-entraînement, le modèle acquiert un niveau significatif de compréhension du langage qui peut ensuite être exploité avec d'autres ensembles de données via un ajustement fin. Ce processus est appelé **apprentissage par transfert**.

![Image tirée de http://jalammar.github.io/illustrated-bert/](../../../../../lessons/5-NLP/18-Transformers/images/jalammarBERT-language-modeling-masked-lm.png)

Il existe de nombreuses variantes des architectures de transformateurs, notamment BERT, DistilBERT, BigBird, OpenGPT3 et bien d'autres, qui peuvent être ajustées. Le [package HuggingFace](https://github.com/huggingface/) fournit un dépôt pour entraîner plusieurs de ces architectures avec PyTorch.

## Utilisation de BERT pour la classification de texte

Voyons comment utiliser un modèle BERT pré-entraîné pour résoudre notre tâche traditionnelle : la classification de séquences. Nous allons classifier notre ensemble de données AG News original.

Tout d'abord, chargeons la bibliothèque HuggingFace et notre ensemble de données :


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Parce que nous allons utiliser un modèle BERT pré-entraîné, nous devrons utiliser un tokenizer spécifique. Tout d'abord, nous allons charger un tokenizer associé au modèle BERT pré-entraîné.

La bibliothèque HuggingFace contient un dépôt de modèles pré-entraînés, que vous pouvez utiliser simplement en spécifiant leurs noms comme arguments dans les fonctions `from_pretrained`. Tous les fichiers binaires nécessaires pour le modèle seront automatiquement téléchargés.

Cependant, dans certains cas, vous devrez charger vos propres modèles. Dans ce cas, vous pouvez spécifier le répertoire contenant tous les fichiers pertinents, y compris les paramètres pour le tokenizer, le fichier `config.json` avec les paramètres du modèle, les poids binaires, etc.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

L'objet `tokenizer` contient la fonction `encode` qui peut être utilisée directement pour encoder du texte :


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Ensuite, créons des itérateurs que nous utiliserons pendant l'entraînement pour accéder aux données. Étant donné que BERT utilise sa propre fonction d'encodage, nous devrons définir une fonction de remplissage similaire à `padify` que nous avons définie auparavant :


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

Dans notre cas, nous utiliserons un modèle BERT pré-entraîné appelé `bert-base-uncased`. Chargeons le modèle en utilisant le package `BertForSequenceClassification`. Cela garantit que notre modèle dispose déjà de l'architecture requise pour la classification, y compris le classificateur final. Vous verrez un message d'avertissement indiquant que les poids du classificateur final ne sont pas initialisés, et que le modèle nécessiterait un pré-entraînement - c'est tout à fait normal, car c'est exactement ce que nous allons faire !


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Nous sommes maintenant prêts à commencer l'entraînement ! Comme BERT est déjà pré-entraîné, nous souhaitons utiliser un taux d'apprentissage relativement faible afin de ne pas altérer les poids initiaux.

Tout le travail important est effectué par le modèle `BertForSequenceClassification`. Lorsque nous appelons le modèle sur les données d'entraînement, il renvoie à la fois la perte (loss) et la sortie du réseau pour le minibatch d'entrée. Nous utilisons la perte pour l'optimisation des paramètres (`loss.backward()` effectue la rétropropagation), et `out` pour calculer la précision de l'entraînement en comparant les étiquettes obtenues `labs` (calculées avec `argmax`) avec les étiquettes attendues `labels`.

Pour contrôler le processus, nous accumulons la perte et la précision sur plusieurs itérations, et nous les affichons tous les `report_freq` cycles d'entraînement.

Cet entraînement prendra probablement beaucoup de temps, donc nous limitons le nombre d'itérations.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Vous pouvez constater (surtout si vous augmentez le nombre d'itérations et attendez suffisamment longtemps) que la classification avec BERT nous donne une précision assez bonne ! Cela s'explique par le fait que BERT comprend déjà très bien la structure de la langue, et que nous n'avons qu'à ajuster le classificateur final. Cependant, comme BERT est un modèle volumineux, tout le processus d'entraînement prend beaucoup de temps et nécessite une puissance de calcul importante ! (GPU, et de préférence plus d'un).

> **Note :** Dans notre exemple, nous utilisons l'un des plus petits modèles BERT pré-entraînés. Il existe des modèles plus grands qui sont susceptibles de donner de meilleurs résultats.


## Évaluation des performances du modèle

Nous pouvons maintenant évaluer les performances de notre modèle sur le jeu de données de test. La boucle d'évaluation est assez similaire à la boucle d'entraînement, mais il ne faut pas oublier de passer le modèle en mode évaluation en appelant `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## À retenir

Dans cette unité, nous avons vu à quel point il est facile de prendre un modèle de langage pré-entraîné de la bibliothèque **transformers** et de l'adapter à notre tâche de classification de texte. De la même manière, les modèles BERT peuvent être utilisés pour l'extraction d'entités, les questions-réponses et d'autres tâches de NLP.

Les modèles de type Transformer représentent l'état de l'art actuel en NLP, et dans la plupart des cas, ils devraient être la première solution avec laquelle vous commencez à expérimenter lorsque vous mettez en œuvre des solutions NLP personnalisées. Cependant, comprendre les principes de base des réseaux neuronaux récurrents discutés dans ce module est extrêmement important si vous souhaitez construire des modèles neuronaux avancés.



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
